# Collocations e Processamento de Comentários de Avaliações de Hotéis

Collocations são duas ou mais palavras que tendem a aparecer frequentemente juntas, como "Estados Unidos", "Rio Grande do Sul" ou "Machine Learning". Essas palavras podem gerar diversas combinações e por isso o contexto também é importante no processamento de linguagem natural.

Os dois tipos mais comuns de Collocations são bigramas e trigramas. Bigramas são duas palavras adjacentes, como "tomografia computadorizada", "aprendizado de máquina" ou "mídia social". Trigramas são três palavras adjacentes, como "fora do negócio" ou "Proctor and Gamble".

- Bigramas: (Nome, Nome), (Adjetivo, Nome)
- Trigramas: (Adjetivo/Nome, Qualquer_Item, Adjetivo/Nome)

Mas se escolhermos palavras adjacentes como bigrama ou trigramas, não obteremos frases significativas. Por exemplo, a frase 'Ele usa mídias sociais' contém bigramas: 'Ele usa', 'usa mídias', 'mídias sociais'. "Ele usa" e "usa mídias" não significa nada, enquanto "mídias sociais" é um bigrama significativo. 

Como fazemos boas seleções para Collocations? As co-ocorrências podem não ser suficientes, pois frases como 'assim como' podem co-ocorrer com frequência, mas não são significativas. Vamos explorar vários métodos para filtrar as Collocations mais significativas: contagem de frequências, informação mútua pontual (PMI) e teste de hipóteses (teste t e qui-quadrado).

### Definição do Problema

Dado um conjunto de texto de avaliações (comentários) de hotéis, vamos buscar as Collocations mais relevantes que ajudam a explicar as avaliações!

In [ ]:
# Se necessário, instale pacotes que não estejam instalados
!pip install -q spacy

In [ ]:
# Imports
import pandas as pd
import nltk
import spacy
import re
import string
from nltk.corpus import stopwords

In [ ]:
# Se necessário, faça o download das stopwords
nltk.download('stopwords')

In [ ]:
# Carregando dados de avaliações de hotéis
# Fonte de dados: https://datafiniti.co/products/business-data/
avaliacoes_hoteis = pd.read_csv('https://raw.githubusercontent.com/dsacademybr/Datasets/master/dataset7.csv')

In [ ]:
# Visualiza os dados
avaliacoes_hoteis.head(5)

In [ ]:
# Tipo do objeto
type(avaliacoes_hoteis)

In [ ]:
# Shape
avaliacoes_hoteis.shape

In [ ]:
# Extraindo as avaliações
comentarios = avaliacoes_hoteis['reviews.text']

In [ ]:
# Converte para o tipo string
comentarios = comentarios.astype('str')

In [ ]:
# Função para remover caracteres non-ascii 
def removeNoAscii(s): 
    return "".join(i for i in s if ord(i) < 128)

In [ ]:
# Remove caracteres non-ascii 
comentarios = comentarios.map(lambda x: removeNoAscii(x))

In [ ]:
# Obtém as stopwords em todos os idiomas
dicionario_stopwords = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}
dicionario_stopwords

In [ ]:
# Função para detectar o idioma predominante com base nas stopwords
def descobre_idioma(text):
    
    # Aplica tokenização considerando pontuação
    palavras = set(nltk.wordpunct_tokenize(text.lower()))
    
    # Conta o total de palavras tokenizadas considerando o dicionário de stopwords
    lang = max(((lang, len(palavras & stopwords)) for lang, stopwords in dicionario_stopwords.items()), key = lambda x: x[1])[0]
    
    # Verifica se o idioma é português
    if lang == 'portuguese':
        return True
    else:
        return False

In [ ]:
# Filtra somente os comentários em português
comentarios_portugues = comentarios[comentarios.apply(descobre_idioma)]

In [ ]:
# Shape
comentarios_portugues.shape

In [ ]:
# Print
comentarios_portugues

In [ ]:
# Função para detectar o idioma predominante com base nas stopwords
def descobre_idioma(text):
    words = set(nltk.wordpunct_tokenize(text.lower()))
    lang = max(((lang, len(words & stopwords)) for lang, stopwords in dicionario_stopwords.items()), key = lambda x: x[1])[0]
    if lang == 'english':
        return True
    else:
        return False

In [ ]:
# Filtra somente os comentários em português
comentarios_ingles = comentarios[comentarios.apply(descobre_idioma)]

In [ ]:
# Shape
comentarios_ingles.shape

In [ ]:
# Print
comentarios_ingles.head()

In [ ]:
# Removendo duplicidades - inplace = True aplica o drop no dataset
comentarios_ingles.drop_duplicates(inplace = True)

In [ ]:
# Shape
comentarios_ingles.shape

In [ ]:
# Baixando o dicionário inglês
# https://spacy.io/usage/models
!python -m spacy download en_core_web_sm

> Pode ser necessário reiniciar o Jupyter Notebook para executar a célula abaixo.

In [ ]:
# Carrega o dcionário em nossa sessão SpaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Função para limpar e lematizar os comentários
def limpa_comentarios(text):
    
    # Remove pontuação usando expressão regular
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", str(text))
    
    # Usa o SpaCy para lematização
    doc = nlp(nopunct, disable = ['parser', 'ner'])
    lemma = [token.lemma_ for token in doc]
    return lemma

In [ ]:
# Aplica a função aos dados
comentarios_ingles_lemmatized = comentarios_ingles.map(limpa_comentarios)

In [ ]:
# Coloca tudo em minúsculo
comentarios_ingles_lemmatized = comentarios_ingles_lemmatized.map(lambda x: [word.lower() for word in x])

In [ ]:
# Visualiza os dados
comentarios_ingles_lemmatized.head()

In [ ]:
# Vamos tokenizar os comentários
comentarios_tokens = [item for items in comentarios_ingles_lemmatized for item in items]

In [ ]:
# Tokens
comentarios_tokens

### Estratégia 1 - Buscando Bigramas e Trigramas Mais Relevantes nos Comentários Por Frequência

Nossa primeira estratégia é a mais simples de todas: contagem de frequência. Contamos quantas vezes cada Collocation aparece no texto e filtramos pelos Collocations mais frequentes.

Vamos filtrar apenas os adjetivos e substantivos para reduzir o tempo de processamento e vamos contar a frequência dos Collocations, bigramas e trigramas, nos comentários.

In [ ]:
# Métricas de associação de bigramas (esse objeto possui diversos atributos, como freq, pmi, teste t, etc...)
bigramas = nltk.collocations.BigramAssocMeasures()

In [ ]:
# Métricas de associação de trigramas
trigramas = nltk.collocations.TrigramAssocMeasures()

In [ ]:
# O próximo passo é criar um buscador de bigramas nos tokens
buscaBigramas = nltk.collocations.BigramCollocationFinder.from_words(comentarios_tokens)

In [ ]:
# Fazemos o mesmo com trigramas. Fique atento aos métodos que estão sendo usados
buscaTrigramas = nltk.collocations.TrigramCollocationFinder.from_words(comentarios_tokens)

In [ ]:
# Vamos contar quantas vezes cada bigrama aparece nos tokens dos comentários
bigrama_freq = buscaBigramas.ngram_fd.items()

In [ ]:
# Frequência dos bigramas
bigrama_freq

In [ ]:
# Vamos converter o dicionário anterior em uma tabela de frequência no formato do Pandas para os bigramas
FreqTabBigramas = pd.DataFrame(list(bigrama_freq), 
                               columns = ['Bigrama', 'Freq']).sort_values(by = 'Freq', ascending = False)

In [ ]:
# Visualiza a tabela
FreqTabBigramas.head(5)

In [ ]:
# Vamos contar quantas vezes cada trigrama aparece nos tokens dos comentários
trigrama_freq = buscaTrigramas.ngram_fd.items()

In [ ]:
# Tabela de frequência no formato do Pandas para os trigramas
FreqTabTrigramas = pd.DataFrame(list(trigrama_freq), 
                                columns = ['Trigrama','Freq']).sort_values(by = 'Freq', ascending = False)

In [ ]:
# Visualiza a tabela
FreqTabTrigramas.head(5)

Temos muitas stopwords. Vamos removê-las.

In [ ]:
# Vamos criar uma lista de stopwords
en_stopwords = set(stopwords.words('english'))

In [ ]:
# Necessário para criar as funções de filtro
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
# Função para filtrar bigramas ADJ/NN e remover stopwords
def filtra_tipo_token_bigrama(ngram):
    
    # Verifica se é pronome
    if '-pron-' in ngram or 't' in ngram:
        return False
    
    # Loop nos ngramas para verificar se é stopword
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
        
    # Tipos de tokens aceitáveis
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Subtipos
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    
    # Tags
    tags = nltk.pos_tag(ngram)
    
    # Retorna o que queremos, ADJ/NN
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
# Agora filtramos os bigramas
bigramas_filtrados = FreqTabBigramas[FreqTabBigramas.Bigrama.map(lambda x: filtra_tipo_token_bigrama(x))]

In [ ]:
# Visualiza a tabela
bigramas_filtrados.head(5)

In [ ]:
# Função para filtrar trigramas ADJ/NN e remover stopwords
def filtra_tipo_token_trigrama(ngram):
    
    # Verifica se é pronome
    if '-pron-' in ngram or 't' in ngram:
        return False
    
    # Loop nos ngramas para verificar se é stopword
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
        
    # Tipos de tokens aceitáveis
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Subtipos
    second_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Tags
    tags = nltk.pos_tag(ngram)
    
    # Retorna o que queremos, ADJ/NN
    if tags[0][1] in first_type and tags[2][1] in second_type:
        return True
    else:
        return False

In [ ]:
# Agora filtramos os trigramas
trigramas_filtrados = FreqTabTrigramas[FreqTabTrigramas.Trigrama.map(lambda x: filtra_tipo_token_trigrama(x))]

In [ ]:
# Visualiza a tabela
trigramas_filtrados.head(5)

Já temos os bigramas e trigramas mais relevantes por frequência. Vamos usar os outros métodos e depois comparar todos eles.

### Estratégia 2 - Buscando Bigramas e Trigramas Mais Relevantes nos Comentários Por PMI

PMI significa Pointwise Mutual Information

PMI é um score que mede a probabilidade com que as palavras co-ocorrem mais do que se fossem independentes. No entanto, é muito sensível à combinação rara de palavras. Por exemplo, se um bigrama aleatório 'abc xyz' aparecer e nem 'abc' nem 'xyz' aparecerem em nenhum outro lugar do texto, 'abc xyz' será identificado como bigrama altamente significativo quando poderia ser apenas um erro ortográfico aleatório ou um frase muito rara para generalizar como um bigrama. Portanto, esse método é frequentemente usado com um filtro de frequência.

In [ ]:
# Vamos retornar somente bigramas com 20 ou mais ocorrências
buscaBigramas.apply_freq_filter(20)

In [ ]:
# Criamos a tabela
PMITabBigramas = pd.DataFrame(list(buscaBigramas.score_ngrams(bigramas.pmi)), 
                              columns = ['Bigrama', 'PMI']).sort_values(by = 'PMI', ascending = False)

In [ ]:
# Visualiza a tabela
PMITabBigramas.head(5)

In [ ]:
# Vamos retornar somente trigramas com 20 ou mais ocorrências
buscaTrigramas.apply_freq_filter(20)

In [ ]:
# Criamos a tabela
PMITabTrigramas = pd.DataFrame(list(buscaTrigramas.score_ngrams(trigramas.pmi)), 
                               columns = ['Trigrama', 'PMI']).sort_values(by = 'PMI', ascending = False)

In [ ]:
# Visualiza a tabela
PMITabTrigramas.head(5)

### Estratégia 3 - Buscando Bigramas e Trigramas Mais Relevantes nos Comentários Por Teste t

O Teste t é um teste estatístico que assume uma distribuição normal dos dados. Na prática, é um teste de hipóteses, uma das bases da Inferência Estatistica.

- H0 é a hipótese nula, que palavras ocorrem em conjunto (bigramas ou trigramas) com determinada probabilidade.
- H1 é a hipótese alternativa, que palavras não ocorrem em conjunto (bigramas ou trigramas) com determinada probabilidade.

Ao aplicar o Teste t rejeitamos ou não a H0 através do cálculo de um score e assim representamos os Collocations mais relevantes no texto.

In [ ]:
# Criamos a tabela para os bigramas
# Observe como o resultado do teste t é obtido: buscaBigramas.score_ngrams(bigramas.student_t)
TestetTabBigramas = pd.DataFrame(list(buscaBigramas.score_ngrams(bigramas.student_t)), 
                             columns = ['Bigrama', 'Teste-t']).sort_values(by = 'Teste-t', ascending = False)

In [ ]:
# Vamos aplicar o filtro pelo tipo de token conforme aplicamos no método 1
bigramas_t_filtrados = TestetTabBigramas[TestetTabBigramas.Bigrama.map(lambda x: filtra_tipo_token_bigrama(x))]

In [ ]:
# Visualiza a tabela
bigramas_t_filtrados.head(5)

In [ ]:
# Criamos a tabela para os trigramas
TestetTabTrigramas = pd.DataFrame(list(buscaTrigramas.score_ngrams(trigramas.student_t)), 
                                  columns = ['Trigrama', 'Teste-t']).sort_values(by = 'Teste-t', ascending = False)

In [ ]:
# Vamos aplicar o filtro pelo tipo de token conforme aplicamos no método 1
trigramas_t_filtrados = TestetTabTrigramas[TestetTabTrigramas.Trigrama.map(lambda x: filtra_tipo_token_trigrama(x))]

In [ ]:
# Visualiza a tabela
trigramas_t_filtrados.head(5)

### Estratégia 4 - Buscando Bigramas e Trigramas Mais Relevantes nos Comentários Por Teste do Qui-quadrado

O teste do qui-quadrado é uma alternativa ao teste t. O teste do qui-quadrado assume na hipótese nula que as palavras são independentes, assim como no teste t.

In [ ]:
# Prepara a tabela
# Observe como estamos coletando a estatística qui-quadrado: buscaBigramas.score_ngrams(bigramas.chi_sq)
QuiTabBigramas = pd.DataFrame(list(buscaBigramas.score_ngrams(bigramas.chi_sq)), 
                              columns = ['Bigrama', 'Qui']).sort_values(by = 'Qui', ascending = False)

In [ ]:
# Visualiza a tabela
QuiTabBigramas.head(5)

In [ ]:
# Prepara a tabela
QuiTabTrigramas = pd.DataFrame(list(buscaTrigramas.score_ngrams(trigramas.chi_sq)), 
                               columns = ['Trigrama', 'Qui']).sort_values(by = 'Qui', ascending = False)

In [ ]:
# Visualiza a tabela
QuiTabTrigramas.head(5)

### Comparação e Resultado Final

In [ ]:
# Vamos extrair os 10 Collocations bigramas mais relevantes de acordo com cada um dos 4 métodos usados
# Lembre-se que aplicamos filtros para remover as stopwords e devemos usar a tabela filtrada
metodo1_bigrama = bigramas_filtrados[:10].Bigrama.values
metodo2_bigrama = PMITabBigramas[:10].Bigrama.values
metodo3_bigrama = bigramas_t_filtrados[:10].Bigrama.values
metodo4_bigrama = QuiTabBigramas[:10].Bigrama.values

In [ ]:
# Vamos extrair os 10 Collocations trigramas mais relevantes de acordo com cada um dos 4 métodos usados
# Lembre-se que aplicamos filtros para remover as stopwords e devemos usar a tabela filtrada
metodo1_trigrama = trigramas_filtrados[:10].Trigrama.values
metodo2_trigrama = PMITabTrigramas[:10].Trigrama.values
metodo3_trigrama = trigramas_t_filtrados[:10].Trigrama.values
metodo4_trigrama = QuiTabTrigramas[:10].Trigrama.values

In [ ]:
# Vamos criar um super dataframe com todos os resultados para bigramas
comparaBigramas = pd.DataFrame([metodo1_bigrama, metodo2_bigrama, metodo3_bigrama, metodo4_bigrama]).T

In [ ]:
# Nossa tabela precisa de nomes para as colunas
comparaBigramas.columns = ['Frequência', 
                           'PMI', 
                           'Teste-t', 
                           'Teste Qui-quadrado']

In [ ]:
# Visualiza a tabela - Padrão CSS
comparaBigramas.style.set_properties(**{'background-color': 'green', 
                                        'color': 'white', 
                                        'border-color': 'white'})

In [ ]:
# Vamos criar um super dataframe com todos os resultados para trigramas
comparaTrigramas = pd.DataFrame([metodo1_trigrama, metodo2_trigrama, metodo3_trigrama, metodo4_trigrama]).T

In [ ]:
# Nossa tabela precisa de nomes para as colunas
comparaTrigramas.columns = ['Frequência', 
                            'PMI', 
                            'Teste-t', 
                            'Teste Qui-quadrado']

In [ ]:
# Visualiza a tabela
comparaTrigramas.style.set_properties(**{'background-color': 'blue', 
                                         'color': 'white', 
                                         'border-color': 'white'})

### Conclusão

Podemos ver que os métodos PMI e Qui-quadrado fornecem bons resultados. Seus resultados também são semelhantes. 

Mas os métodos de Frequência e Teste-t apresentam os melhores resutados e são também muito semelhantes entre si. 

Em aplicativos reais, podemos observar a lista e definir um limite em um valor a partir de quando a lista para de fazer sentido. Também podemos fazer testes diferentes para ver qual lista parece fazer mais sentido para um determinado conjunto de dados. 

Como alternativa, podemos combinar resultados de várias listas. Uma boa escolha é multiplicar o PMI e a Frequência para levar em consideração o aumento da probabilidade e a frequência da ocorrência. Ou multiplicar a frequência pelo Teste-t criando assim um índice único de relevância das Collocations.

Para este trabalho, vamos considerar as Collocations calculadas por Frequência como as mais relevantes. A escolha se deve ao fato de que as suposições para o Teste-t não foram validadas e usar seu resultado seria inadequado. Salvamos a coluna de frequência do dataframe final em formato csv ou txt e encaminhamos à área de Marketing da rede de hotéis.

Você pode continuar o trabalho e alterar os métodos acima, se desejar.

# Fim